1) С помощью BeautifulSoup спарсить новости с https://news.yandex.ru по своему региону.  

* Заголовок   
* Краткое описание   
* Ссылка на новость   

2) Разбить новости по категориям  
3) Расположить в хронологическом порядке  

In [1]:
import requests
from bs4 import BeautifulSoup
import datetime
# для понимания русскоязычного названия месяца
import locale

In [2]:
# Возвращает результат запроса
def html_from_url(url, params, timeout=50):
    msg = ''
    response = requests.Response()
    try:
        headers = {'accept': '*/&*',
                   'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                                 '(KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'}
        response = requests.get(url, params=params, headers=headers, timeout=timeout)
        print(response.url)
    except requests.exceptions.ConnectionError:
        msg = 'Проблемы с интернет соединением'
    except requests.RequestException as e:
        msg = str(e)
    return response.text, response.status_code, msg

#не самое лучше решение для идентификации месяца, но за отсутствием времени...
def month(s_month):
    d_month = {'января':'Январь', 
               'февраля':'Февраль', 
               'марта':'Март', 
               'апреля':'Апрель', 
               'мая':'Май', 
               'июня':'Июнь', 
               'июля':'Июль', 
               'августа':'Август', 
               'сентября':'Сентябрь', 
               'октября':'Октябрь', 
               'ноября':'Ноябрь', 
               'декабря':'Декабрь'
              }
    if d_month[s_month] != None:
        return d_month[s_month]
    else:
        return s_month
    
#Парсинг даты обновления новости    
def get_date_from_story_date(s_time):
    try:        
        if s_time.find('Обновлено в') >= 0:
            d = datetime.date.today()
            t = datetime.datetime.strptime(s_time, 'Обновлено в %H:%M').time()
            news_time = datetime.datetime.combine(d, t) 
        elif s_time.find('вчера') >= 0:
            d = datetime.date.today() - datetime.timedelta(days = 1)
            t = datetime.datetime.strptime(s_time, 'Обновлено вчера в %H:%M').time()
            news_time = datetime.datetime.combine(d, t) 
        elif s_time.find('Обновлено') >= 0:
            s_date = str(s_time[11:]).replace('\xa0', ' ')
            
            #приведение месяца к именительному падежу
            spam = s_date.find(' в ')
            if  spam>=0:
                s_month = s_date[3:spam]
                s_date = s_date.replace(s_month, month(s_month))
                
            s_date = s_date.replace(' в ',f' {str((datetime.date.today()).year)} ')                    
            locale.setlocale(locale.LC_ALL, 'ru_RU.UTF-8')
            news_time = datetime.datetime.strptime(s_date, f'%d %B %Y %H:%M')
        else:
            news_time = s_time
    except ValueError:
            news_time = s_time            
    return news_time

def parse_news(url, city):
    news_url = url+'/'+city
    html_doc, status_code, msg = html_from_url(news_url, '')
    result = []
    if status_code != 200:
        print(f'Ошибка загрузки страницы {news_url}: {msg}')
    else:
        soup = BeautifulSoup(html_doc, 'html.parser')    
        news = soup.findAll('div', {'class': 'story'})
        for one_news in news:
            title = one_news.find('a', {'class': 'link_theme_black'}).string
            try:
                story_text = one_news.find('div', {'class': 'story__text'}).string
            except AttributeError:
                story_text = ''
            href = f"{url}{one_news.find('a', {'class': 'link_theme_black'})['href']}"
            try: 
                category =  one_news.find('a', {'class': 'rubric-label'}).text
            except AttributeError:
                category = ''
            s_time = one_news.find('div', {'class': 'story__date'}).text.strip()
            story_time = get_date_from_story_date(s_time)
            result.append(
            {
                'title':title,
                'story_text':story_text,
                'href':href,
                'category':category,
                'story_time':story_time
            })
    return result

#Получить список категорий
def get_categories(news):
    categories = set()
    for one_news in news:
        categories.add(one_news['category'])
    return categories

def print_news(news):
    print(news['title'])
    print(news['category'])
    print(news['story_text'])
    print(news['href'])
    print(news['story_time'])
    print('*'*100)
    

In [3]:
city = 'Perm'
url = 'https://news.yandex.ru'
news = parse_news(url, city)
for cat in get_categories(news):
    print(f'{"-"*40}{cat}{"-"*40}')
    news_category = filter(lambda x : x['category'] == cat, news)
    for one_news in sorted(news_category, key=lambda x: x['story_time'] ):
        print_news(one_news)

https://news.yandex.ru/Perm
----------------------------------------ЖКХ----------------------------------------
На следующей неделе на сутки без воды останутся несколько микрорайонов Перми
ЖКХ
На следующей неделе «НОВОГОР-Прикамье» проведет плановые ремонтные работы по адресу ул. Куйбышева,109.
https://news.yandex.ru/story/Na_sleduyushhej_nedele_na_sutki_bez_vody_ostanutsya_neskolko_mikrorajonov_Permi--af3638a9b74e78b7ac1f002c1b465ff4?lr=50&lang=ru&stid=d6WbTc-N&persistent_id=69949720&rubric=Perm&from=region
2019-07-30 20:05:00
****************************************************************************************************
На Комсомольском проспекте появятся люки со звериным стилем
ЖКХ
На тротуарах и бульварной части Комсомольского проспекта в Перми будут установлены канализационные люки с гравировкой звериного стиля.
https://news.yandex.ru/story/Na_Komsomolskom_prospekte_poyavyatsya_lyuki_so_zverinym_stilem--81595920a01dad0325397e706dada4d1?lr=50&lang=ru&stid=XhEAjiatSf7tiFpi8K5r&